Every line is separated by '\<eos>' tag and a ',' comma. Also each sonnet's beginning is represented by '\\' symbol or by counting 14 lines from initial sonnet

In [1]:
# Loading Dataset
f = open('dataset.txt', 'r', encoding='utf-8')
sonnets_dataset = f.read().split('<eos>')
del sonnets_dataset[-1]

In [2]:
# Total number of sonnets in dataset
len_sonnets = int(len(sonnets_dataset)/14)
print("Total Number of Sonnets in dataset:", len_sonnets)

Total Number of Sonnets in dataset: 2685


In [3]:
# To get each sonnet separately
def sonnet_print(sonnet):
    return sonnets_dataset[sonnet:sonnet+14]

for sonnet in range(0, 2):
    _ = sonnet_print(sonnet)
    print("Sonnet:", sonnet, "\n", _, "\n")   

Sonnet: 0 
 ['with clytia he no longer was received ', ' than while he was a man of wealth believed ', " balls , concerts , op'ras , tournaments , and plays ", ' expensive dresses , all engaging ways ', ' were used to captivate this lady fair ', ' while scarcely one around but in despair ', ' wife , widow , maid , his fond affection sought ', " to gain him , ev'ry wily art was brought ", " but all in vain :— by passion overpow'red ", ' the belle , whose conduct others would have soured ', ' to him appeared a goddess full of charms ', " superior e'en to helen , in his arms ", ' from whence we may conclude , the beauteous dame ', " was always deaf to fred'rick ' s ardent flame "] 

Sonnet: 1 
 [' than while he was a man of wealth believed ', " balls , concerts , op'ras , tournaments , and plays ", ' expensive dresses , all engaging ways ', ' were used to captivate this lady fair ', ' while scarcely one around but in despair ', ' wife , widow , maid , his fond affection sought ', " to gai

In [4]:
# Loading Libraries
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [5]:
sonnets_dataset

['with clytia he no longer was received ',
 ' than while he was a man of wealth believed ',
 " balls , concerts , op'ras , tournaments , and plays ",
 ' expensive dresses , all engaging ways ',
 ' were used to captivate this lady fair ',
 ' while scarcely one around but in despair ',
 ' wife , widow , maid , his fond affection sought ',
 " to gain him , ev'ry wily art was brought ",
 " but all in vain :— by passion overpow'red ",
 ' the belle , whose conduct others would have soured ',
 ' to him appeared a goddess full of charms ',
 " superior e'en to helen , in his arms ",
 ' from whence we may conclude , the beauteous dame ',
 " was always deaf to fred'rick ' s ardent flame ",
 '\nour love is not a fading earthly flower ',
 ' its wing è d seed dropped down from paradise ',
 ' and , nursed by day and night , by sun and shower ',
 ' doth momently to fresher beauty rise ',
 ' to us the leafless autumn is not bare ',
 " nor winter 's rattling boughs lack lusty green ",
 " our summer hear

In [6]:
dataset_string = ''
for line in sonnets_dataset:
    dataset_string = dataset_string + line
dataset_string[:500]

"with clytia he no longer was received  than while he was a man of wealth believed  balls , concerts , op'ras , tournaments , and plays  expensive dresses , all engaging ways  were used to captivate this lady fair  while scarcely one around but in despair  wife , widow , maid , his fond affection sought  to gain him , ev'ry wily art was brought  but all in vain :— by passion overpow'red  the belle , whose conduct others would have soured  to him appeared a goddess full of charms  superior e'en to"

In [7]:
# Encoding Dataset 
chars = tuple(set(dataset_string))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in dataset_string])
encoded[:100]

array([17, 55, 50, 36, 82, 81, 63, 72, 50, 55, 46, 82, 36, 15, 82, 65, 35,
       82, 63, 35, 65,  7, 15, 39, 82, 17, 46, 47, 82, 39, 15, 81, 15, 55,
       77, 15, 79, 82, 82, 50, 36, 46, 65, 82, 17, 36, 55, 63, 15, 82, 36,
       15, 82, 17, 46, 47, 82, 46, 82, 13, 46, 65, 82, 35, 76, 82, 17, 15,
       46, 63, 50, 36, 82, 26, 15, 63, 55, 15, 77, 15, 79, 82, 82, 26, 46,
       63, 63, 47, 82,  3, 82, 81, 35, 65, 81, 15, 39, 50, 47, 82])

In [8]:
dataset_string[618:619], encoded[618:619]

('\n', array([53]))

In [9]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [10]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [11]:
batches = get_batches(encoded, 8, 14)
x, y = next(batches)

In [12]:
x[:2]

array([[17, 55, 50, 36, 82, 81, 63, 72, 50, 55, 46, 82, 36, 15],
       [39, 55, 13, 46, 65, 79, 82, 36, 55, 13, 82, 47, 35, 73]])

In [13]:
y[:2]

array([[55, 50, 36, 82, 81, 63, 72, 50, 55, 46, 82, 36, 15, 82],
       [55, 13, 46, 65, 79, 82, 36, 55, 13, 82, 47, 35, 73, 65]])

In [14]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [15]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [16]:
def train(net, data, epochs=10, batch_size=10, seq_length=14, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [17]:
# define and print the net
n_hidden=512
n_layers=3

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(88, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=88, bias=True)
)


In [18]:
batch_size = 32
seq_length = 14
n_epochs = 10 # 

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/10... Step: 10... Loss: 3.0620... Val Loss: 2.9893
Epoch: 1/10... Step: 20... Loss: 2.9145... Val Loss: 2.8913
Epoch: 1/10... Step: 30... Loss: 2.9466... Val Loss: 2.8730
Epoch: 1/10... Step: 40... Loss: 2.9529... Val Loss: 2.8711
Epoch: 1/10... Step: 50... Loss: 2.9036... Val Loss: 2.8697
Epoch: 1/10... Step: 60... Loss: 2.8860... Val Loss: 2.8748
Epoch: 1/10... Step: 70... Loss: 2.9109... Val Loss: 2.8709
Epoch: 1/10... Step: 80... Loss: 2.8297... Val Loss: 2.8676
Epoch: 1/10... Step: 90... Loss: 2.8946... Val Loss: 2.8680
Epoch: 1/10... Step: 100... Loss: 2.8449... Val Loss: 2.8670
Epoch: 1/10... Step: 110... Loss: 2.8378... Val Loss: 2.8639
Epoch: 1/10... Step: 120... Loss: 2.8866... Val Loss: 2.8654
Epoch: 1/10... Step: 130... Loss: 2.8444... Val Loss: 2.8678
Epoch: 1/10... Step: 140... Loss: 2.8898... Val Loss: 2.8651
Epoch: 1/10... Step: 150... Loss: 2.8986... Val Loss: 2.8642
Epoch: 1/10... Step: 160... Loss: 2.8027... Val Loss: 2.8629
Epoch: 1/10... Step: 170... Loss:

Epoch: 1/10... Step: 1350... Loss: 1.8461... Val Loss: 1.8103
Epoch: 1/10... Step: 1360... Loss: 1.9154... Val Loss: 1.8101
Epoch: 1/10... Step: 1370... Loss: 1.9052... Val Loss: 1.8042
Epoch: 1/10... Step: 1380... Loss: 1.9153... Val Loss: 1.8066
Epoch: 1/10... Step: 1390... Loss: 1.8854... Val Loss: 1.8038
Epoch: 1/10... Step: 1400... Loss: 1.7867... Val Loss: 1.8126
Epoch: 1/10... Step: 1410... Loss: 1.8080... Val Loss: 1.8022
Epoch: 1/10... Step: 1420... Loss: 1.8485... Val Loss: 1.7989
Epoch: 1/10... Step: 1430... Loss: 1.8677... Val Loss: 1.7942
Epoch: 1/10... Step: 1440... Loss: 1.8153... Val Loss: 1.7938
Epoch: 1/10... Step: 1450... Loss: 1.8307... Val Loss: 1.7921
Epoch: 1/10... Step: 1460... Loss: 1.8426... Val Loss: 1.7882
Epoch: 1/10... Step: 1470... Loss: 1.7530... Val Loss: 1.7859
Epoch: 1/10... Step: 1480... Loss: 1.8433... Val Loss: 1.7818
Epoch: 1/10... Step: 1490... Loss: 1.8143... Val Loss: 1.7826
Epoch: 1/10... Step: 1500... Loss: 1.8948... Val Loss: 1.7824
Epoch: 1

Epoch: 1/10... Step: 2680... Loss: 1.7232... Val Loss: 1.6309
Epoch: 1/10... Step: 2690... Loss: 1.8763... Val Loss: 1.6335
Epoch: 1/10... Step: 2700... Loss: 1.6934... Val Loss: 1.6329
Epoch: 1/10... Step: 2710... Loss: 1.7913... Val Loss: 1.6320
Epoch: 1/10... Step: 2720... Loss: 1.6125... Val Loss: 1.6298
Epoch: 1/10... Step: 2730... Loss: 1.6754... Val Loss: 1.6300
Epoch: 1/10... Step: 2740... Loss: 1.6196... Val Loss: 1.6261
Epoch: 1/10... Step: 2750... Loss: 1.8452... Val Loss: 1.6263
Epoch: 1/10... Step: 2760... Loss: 1.7670... Val Loss: 1.6264
Epoch: 1/10... Step: 2770... Loss: 1.7199... Val Loss: 1.6238
Epoch: 1/10... Step: 2780... Loss: 1.7011... Val Loss: 1.6248
Epoch: 1/10... Step: 2790... Loss: 1.7454... Val Loss: 1.6240
Epoch: 1/10... Step: 2800... Loss: 1.7019... Val Loss: 1.6244
Epoch: 1/10... Step: 2810... Loss: 1.8112... Val Loss: 1.6206
Epoch: 1/10... Step: 2820... Loss: 1.7702... Val Loss: 1.6192
Epoch: 1/10... Step: 2830... Loss: 1.6636... Val Loss: 1.6168
Epoch: 1

Epoch: 2/10... Step: 4010... Loss: 1.7123... Val Loss: 1.5568
Epoch: 2/10... Step: 4020... Loss: 1.5681... Val Loss: 1.5537
Epoch: 2/10... Step: 4030... Loss: 1.5485... Val Loss: 1.5557
Epoch: 2/10... Step: 4040... Loss: 1.6141... Val Loss: 1.5537
Epoch: 2/10... Step: 4050... Loss: 1.6236... Val Loss: 1.5488
Epoch: 2/10... Step: 4060... Loss: 1.6527... Val Loss: 1.5494
Epoch: 2/10... Step: 4070... Loss: 1.5489... Val Loss: 1.5501
Epoch: 2/10... Step: 4080... Loss: 1.4768... Val Loss: 1.5502
Epoch: 2/10... Step: 4090... Loss: 1.6284... Val Loss: 1.5510
Epoch: 2/10... Step: 4100... Loss: 1.6803... Val Loss: 1.5479
Epoch: 2/10... Step: 4110... Loss: 1.7180... Val Loss: 1.5485
Epoch: 2/10... Step: 4120... Loss: 1.6773... Val Loss: 1.5563
Epoch: 2/10... Step: 4130... Loss: 1.6969... Val Loss: 1.5513
Epoch: 2/10... Step: 4140... Loss: 1.6974... Val Loss: 1.5481
Epoch: 2/10... Step: 4150... Loss: 1.8375... Val Loss: 1.5474
Epoch: 2/10... Step: 4160... Loss: 1.8070... Val Loss: 1.5488
Epoch: 2

Epoch: 2/10... Step: 5340... Loss: 1.5021... Val Loss: 1.5019
Epoch: 2/10... Step: 5350... Loss: 1.6344... Val Loss: 1.5018
Epoch: 2/10... Step: 5360... Loss: 1.3942... Val Loss: 1.5024
Epoch: 2/10... Step: 5370... Loss: 1.5887... Val Loss: 1.5042
Epoch: 2/10... Step: 5380... Loss: 1.5467... Val Loss: 1.5027
Epoch: 2/10... Step: 5390... Loss: 1.5983... Val Loss: 1.5008
Epoch: 2/10... Step: 5400... Loss: 1.5408... Val Loss: 1.5034
Epoch: 2/10... Step: 5410... Loss: 1.6340... Val Loss: 1.5033
Epoch: 2/10... Step: 5420... Loss: 1.4741... Val Loss: 1.5026
Epoch: 2/10... Step: 5430... Loss: 1.5227... Val Loss: 1.5017
Epoch: 2/10... Step: 5440... Loss: 1.5329... Val Loss: 1.5032
Epoch: 2/10... Step: 5450... Loss: 1.6273... Val Loss: 1.4983
Epoch: 2/10... Step: 5460... Loss: 1.5541... Val Loss: 1.5000
Epoch: 2/10... Step: 5470... Loss: 1.4985... Val Loss: 1.4995
Epoch: 2/10... Step: 5480... Loss: 1.5526... Val Loss: 1.4988
Epoch: 2/10... Step: 5490... Loss: 1.5650... Val Loss: 1.5021
Epoch: 2

...

Epoch: 9/10... Step: 27680... Loss: 1.4876... Val Loss: 1.3424
Epoch: 9/10... Step: 27690... Loss: 1.4727... Val Loss: 1.3414
Epoch: 9/10... Step: 27700... Loss: 1.4514... Val Loss: 1.3418
Epoch: 9/10... Step: 27710... Loss: 1.3780... Val Loss: 1.3419
Epoch: 9/10... Step: 27720... Loss: 1.3576... Val Loss: 1.3428
Epoch: 9/10... Step: 27730... Loss: 1.3951... Val Loss: 1.3394
Epoch: 9/10... Step: 27740... Loss: 1.4497... Val Loss: 1.3398
Epoch: 9/10... Step: 27750... Loss: 1.2747... Val Loss: 1.3413
Epoch: 9/10... Step: 27760... Loss: 1.3940... Val Loss: 1.3414
Epoch: 9/10... Step: 27770... Loss: 1.4108... Val Loss: 1.3414
Epoch: 9/10... Step: 27780... Loss: 1.4105... Val Loss: 1.3417
Epoch: 9/10... Step: 27790... Loss: 1.4644... Val Loss: 1.3419
Epoch: 9/10... Step: 27800... Loss: 1.4931... Val Loss: 1.3423
Epoch: 9/10... Step: 27810... Loss: 1.4400... Val Loss: 1.3399
Epoch: 9/10... Step: 27820... Loss: 1.3806... Val Loss: 1.3401
Epoch: 9/10... Step: 27830... Loss: 1.4271... Val Loss:

Epoch: 9/10... Step: 28990... Loss: 1.4489... Val Loss: 1.3370
Epoch: 9/10... Step: 29000... Loss: 1.4231... Val Loss: 1.3357
Epoch: 9/10... Step: 29010... Loss: 1.3512... Val Loss: 1.3347
Epoch: 9/10... Step: 29020... Loss: 1.4256... Val Loss: 1.3355
Epoch: 9/10... Step: 29030... Loss: 1.3390... Val Loss: 1.3363
Epoch: 9/10... Step: 29040... Loss: 1.5013... Val Loss: 1.3365
Epoch: 9/10... Step: 29050... Loss: 1.3778... Val Loss: 1.3362
Epoch: 9/10... Step: 29060... Loss: 1.3800... Val Loss: 1.3367
Epoch: 9/10... Step: 29070... Loss: 1.4342... Val Loss: 1.3385
Epoch: 9/10... Step: 29080... Loss: 1.3852... Val Loss: 1.3382
Epoch: 9/10... Step: 29090... Loss: 1.4782... Val Loss: 1.3367
Epoch: 9/10... Step: 29100... Loss: 1.4504... Val Loss: 1.3370
Epoch: 9/10... Step: 29110... Loss: 1.4617... Val Loss: 1.3375
Epoch: 9/10... Step: 29120... Loss: 1.4776... Val Loss: 1.3362
Epoch: 9/10... Step: 29130... Loss: 1.4931... Val Loss: 1.3356
Epoch: 9/10... Step: 29140... Loss: 1.3966... Val Loss:

Epoch: 9/10... Step: 30300... Loss: 1.3870... Val Loss: 1.3370
Epoch: 9/10... Step: 30310... Loss: 1.4152... Val Loss: 1.3375
Epoch: 9/10... Step: 30320... Loss: 1.3417... Val Loss: 1.3370
Epoch: 9/10... Step: 30330... Loss: 1.5043... Val Loss: 1.3381
Epoch: 9/10... Step: 30340... Loss: 1.5020... Val Loss: 1.3390
Epoch: 9/10... Step: 30350... Loss: 1.3366... Val Loss: 1.3380
Epoch: 9/10... Step: 30360... Loss: 1.4550... Val Loss: 1.3381
Epoch: 9/10... Step: 30370... Loss: 1.4001... Val Loss: 1.3398
Epoch: 9/10... Step: 30380... Loss: 1.3991... Val Loss: 1.3411
Epoch: 9/10... Step: 30390... Loss: 1.5323... Val Loss: 1.3389
Epoch: 9/10... Step: 30400... Loss: 1.3840... Val Loss: 1.3385
Epoch: 9/10... Step: 30410... Loss: 1.4145... Val Loss: 1.3381
Epoch: 9/10... Step: 30420... Loss: 1.4573... Val Loss: 1.3372
Epoch: 10/10... Step: 30430... Loss: 1.6361... Val Loss: 1.3371
Epoch: 10/10... Step: 30440... Loss: 1.3293... Val Loss: 1.3388
Epoch: 10/10... Step: 30450... Loss: 1.2585... Val Lo

Epoch: 10/10... Step: 31590... Loss: 1.3373... Val Loss: 1.3364
Epoch: 10/10... Step: 31600... Loss: 1.3471... Val Loss: 1.3363
Epoch: 10/10... Step: 31610... Loss: 1.3650... Val Loss: 1.3359
Epoch: 10/10... Step: 31620... Loss: 1.4061... Val Loss: 1.3352
Epoch: 10/10... Step: 31630... Loss: 1.3700... Val Loss: 1.3351
Epoch: 10/10... Step: 31640... Loss: 1.4881... Val Loss: 1.3350
Epoch: 10/10... Step: 31650... Loss: 1.3237... Val Loss: 1.3363
Epoch: 10/10... Step: 31660... Loss: 1.4668... Val Loss: 1.3369
Epoch: 10/10... Step: 31670... Loss: 1.4303... Val Loss: 1.3356
Epoch: 10/10... Step: 31680... Loss: 1.4201... Val Loss: 1.3360
Epoch: 10/10... Step: 31690... Loss: 1.4062... Val Loss: 1.3371
Epoch: 10/10... Step: 31700... Loss: 1.4181... Val Loss: 1.3372
Epoch: 10/10... Step: 31710... Loss: 1.3965... Val Loss: 1.3374
Epoch: 10/10... Step: 31720... Loss: 1.4006... Val Loss: 1.3369
Epoch: 10/10... Step: 31730... Loss: 1.5370... Val Loss: 1.3355
Epoch: 10/10... Step: 31740... Loss: 1.3

Epoch: 10/10... Step: 32880... Loss: 1.5012... Val Loss: 1.3325
Epoch: 10/10... Step: 32890... Loss: 1.4250... Val Loss: 1.3305
Epoch: 10/10... Step: 32900... Loss: 1.4419... Val Loss: 1.3290
Epoch: 10/10... Step: 32910... Loss: 1.3974... Val Loss: 1.3296
Epoch: 10/10... Step: 32920... Loss: 1.3941... Val Loss: 1.3309
Epoch: 10/10... Step: 32930... Loss: 1.3297... Val Loss: 1.3313
Epoch: 10/10... Step: 32940... Loss: 1.3218... Val Loss: 1.3320
Epoch: 10/10... Step: 32950... Loss: 1.3685... Val Loss: 1.3334
Epoch: 10/10... Step: 32960... Loss: 1.4843... Val Loss: 1.3337
Epoch: 10/10... Step: 32970... Loss: 1.4074... Val Loss: 1.3306
Epoch: 10/10... Step: 32980... Loss: 1.3301... Val Loss: 1.3308
Epoch: 10/10... Step: 32990... Loss: 1.4775... Val Loss: 1.3308
Epoch: 10/10... Step: 33000... Loss: 1.3750... Val Loss: 1.3336
Epoch: 10/10... Step: 33010... Loss: 1.3679... Val Loss: 1.3337
Epoch: 10/10... Step: 33020... Loss: 1.3986... Val Loss: 1.3326
Epoch: 10/10... Step: 33030... Loss: 1.2

In [19]:
# change the name, for saving multiple files
model_name = 'rnn_10epochs.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [37]:
#Saving the whole model
torch.save(net, 'model1.pt')
# torch.save(net.state_dict(), 'model_try.pt')

In [21]:
#Loading the whole Model
# net.load_state_dict(torch.load('model.pt'))
# net.eval()

#Loading the whole Model
net = torch.load('model1.pt')

CharRNN(
  (lstm): LSTM(88, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=88, bias=True)
)

In [22]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [23]:
def sample(net, size, prime='whose conduct others', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [29]:
print(sample(net, 1000, prime='thou wert', top_k=5))

thou wert , and fall  and with the last and flattering treasure  and life the shore of mortal hairly love , i see  all thick and leaves that may as thought and friend  then she well would i spoke i love a silent way  when thou thou hast i shall the world and thee  the blushing show , who without his soften flames  are woo 'd them to a well to take the sea  the foe of the figure to thy love  and so , and though the work that strong and faire  some with thy honey far whose fatal world in morning first  and with a stone of hearts  with strains through the life and to be but i see  and she , and tenth thee to her home that stands  the sense to tress of married face  so warry , so , thy house as if though in the best  to live , a sight , and with my serfect , and which i seem 'd  the wheat thoughes to show the base  to take the steel of worship to his hour  that truth , and light and fair  only i lave , and then , as in the soul and flood  of the leaf , when they this bright looks , that sh

In [27]:
print(sample(net, 1000, prime='who are', top_k=5))

who are  to see the lighter , which i waked as sight  the soul with selfe as there , the latest lord  the barbeds , satisfied a troubled borrows , shall not far  with streams that wild by signes shares the love  when in strength of such sole strength , a praise  the worth , in shore , to love and truth and spread  the bounds of love , thou spoke i would not be to see  so were went and strong to the flowers of storm  still to a stain of meane , when i as the beauty of the sun 
when shall to be a frost thing  the sea with those thou all thou art and true  who lie with sights the sun  the still their fleases of my love and star and breath  and who , and love in some a many fate  who with this shell  that i the book , and walls to see  would not supplied in thee , they to my fame , but i said , startled with the late  of sudden friend , before the stray  she held them woman 's world too said  trembles , till high to listening the sun , the live  the word of that so meaning  then too me sta